In [61]:
import csv
from collections import defaultdict
from datetime import  datetime
from scipy.io import arff
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statistics as stat
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt

import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors
from collections import defaultdict
from collections import Counter
from datetime import  datetime

%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15,5)

创建并且把配对储存为CSV

In [27]:

def csv_dict_write(path,head,data):
    with open(path,'w',encoding='utf-8',newline='') as f:
        writer = csv.DictWriter(f,head)
        writer.writeheader()
        writer.writerows(data)
        return True


In [28]:

def csv_dict_read(path):
    f=open(path,'r',encoding='utf-8')
    reader = csv.DictReader(f,dialect='excel')
    return reader


In [52]:
def get_lable(path):
    reader = csv_dict_read(path)
    data=[]
    LDi_list=defaultdict(list)
    comment_ID_list=defaultdict(list)
    comment_time=defaultdict(list)
    comment_timedata=defaultdict(list)
    commer_per_list=defaultdict(list)
    time_gap=0
    time_gap_list=defaultdict(list)
    head=['issue_no','comment_ID_list','LDi_list','comment_time','commer_per_list','time_gap','time_gap_list','LD1','LD2','LD3','LD4','LD5','LD6','N','total']
    for row in reader:
        try:
            time=datetime.strptime(row['time'],'%Y-%m-%dT%H:%M:%SZ')
        except:
            pass
        LDi_list[row['issue_no']].append(row['pred_label_y'])
        comment_ID_list[row['issue_no']].append(row['comment_ID'])
        comment_time[row['issue_no']].append(row['time'])
        comment_timedata[row['issue_no']].append(time)
    for i,j in LDi_list.items():
        if len(j)>1:
            try:
                time_gap=(comment_timedata[i][-1]-comment_timedata[i][0]).seconds
            except:
                time_gap='bad'
            for x in range(1,len(j)):
                commer_per_list[i].append([j[x-1],j[x]])
                try:
                    time_gap_list[i].append((comment_timedata[i][x]-comment_timedata[i][x-1]).seconds)
                except:
                     time_gap_list[i].append('bad')
        data.append({'issue_no':i,
        'comment_ID_list':comment_ID_list[i],
        'LDi_list':LDi_list[i],
        'comment_time':comment_time[i],
        'commer_per_list':commer_per_list[i],
        'time_gap':time_gap,
        'time_gap_list':time_gap_list[i],
        'LD1':j.count('LD1'),
        'LD2':j.count('LD2'),
        'LD3':j.count('LD3'),
        'LD4':j.count('LD4'),
        'LD5':j.count('LD5'),
        'LD6':j.count('LD6'),
        'N':j.count('N'),
        'total':len(j)})
    csv_dict_write('lab\\'+path[:-4]+'_lab'+'.CSV',head,data)


In [30]:
def creat_lable():
    get_lable('new_atom.CSV')
    get_lable('new_bitcoin.CSV')
    get_lable('new_brew.CSV')
    get_lable('new_ember.CSV')
    get_lable('new_scikit-learn.CSV')

In [54]:
#creat_lable()

In [56]:
atom_lab=pd.read_csv('lab\\new_atom_lab.CSV')
bitcoin_lab=pd.read_csv('lab\\new_bitcoin_lab.CSV')
brew_lab=pd.read_csv('lab\\new_brew_lab.CSV')
ember_lab=pd.read_csv('lab\\new_ember_lab.CSV')
s_learn_lab=pd.read_csv('lab\\new_scikit-learn_lab.CSV')
date_list=[atom_lab,bitcoin_lab,brew_lab,ember_lab,s_learn_lab]

In [79]:
a=atom_lab.commer_per_list.to_list()
a=list(a)
for i in range(len(a)):
    a[i]=a[i][2:-2].split('], [')

a[1][0]

"'N', 'N'"

In [108]:
b=list()
for i in a:
    for l in i:
        if len(l)>4:
            l=l[1:-1].split("', '")
            l=l[0]+l[1]
            l=l.replace('LD','')
        b.append(l)


In [117]:
dict(Counter(b))

{'': 6596,
 'NN': 13551,
 'N3': 3232,
 '45': 415,
 'N6': 297,
 '6N': 309,
 'N5': 1091,
 'N1': 1026,
 '1N': 1215,
 '3N': 3361,
 '53': 205,
 '5N': 595,
 'N2': 667,
 '2N': 806,
 'N4': 1890,
 '4N': 2575,
 '14': 226,
 '44': 645,
 '46': 55,
 '51': 58,
 '41': 278,
 '11': 167,
 '15': 162,
 '35': 332,
 '22': 97,
 '32': 284,
 '23': 188,
 '43': 717,
 '34': 527,
 '33': 1597,
 '31': 270,
 '55': 89,
 '63': 36,
 '13': 310,
 '26': 12,
 '62': 17,
 '12': 64,
 '16': 31,
 '54': 89,
 '42': 193,
 '64': 42,
 '36': 66,
 '56': 18,
 '65': 23,
 '24': 137,
 '25': 104,
 '61': 20,
 '66': 17,
 '21': 63,
 '52': 37,
 'N': 25,
 '3': 11,
 '4': 4,
 '1': 2,
 '2': 3,
 '5': 2}

In [155]:
def get_pair(data):
    a=data.commer_per_list.to_list()
    a=list(a)
    for i in range(len(a)):
        a[i]=a[i][2:-2].split('], [')
    b=list()
    for i in a:
        for l in i:
            if len(l)>4:
                l=l[1:-1].split("', '")
                l=l[0]+l[1]
                l=l.replace('LD','')
            b.append(l)
    v={
    'pair':list(dict(Counter(b)).keys()),
    'number':list(dict(Counter(b)).values())
    }
    result=pd.DataFrame(data=v)
    return result

,number
count,56.000000
mean,800.875000
std,2059.055053
min,2.000000
25%,36.750000
50%,177.500000
75%,607.500000
max,13551.000000
